In [6]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

import sys
sys.path.append("../")

In [7]:
data = pd.read_csv("monte_carlo.csv")

In [8]:
class NashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)
        p = np.zeros((2,2,n))

        for a_j in [0,1]:
            util1 = np.dot(self.exog[:,0:k], params[0:k])  + params[k]*a_j
            util2 = np.dot(self.exog[:,k:], params[k+1:2*k+1]) + params[2*k+1]*a_j
            p[0,a_j,:] = 1 /(1 + np.exp(util1))
            p[1,a_j,:] = 1 /(1 + np.exp(util2))
        
        lamb = .5
        
        #solve for probablity of nash
        mult_eq = np.maximum( ( p[0,1,:]  - p[0,0,:] )*(  p[1,1,:] - p[1,0,:] ),0)
        prob01 = ( p[0,1,:] )*( 1 - p[1,0,:] ) - (1-lamb)*mult_eq
        prob10 = ( 1 - p[0,0,:] )*( p[1,1,:] ) - (lamb)*mult_eq
        prob00 = p[0,0,:] * p[1,0,:]
        prob11 = ( 1 - p[0,1,:] )*( 1 - p[1,1,:] )

        #compute empirical likelihood
        p00 = (1 - self.endog[:,0])*(1 - self.endog[:,1])
        p11 = self.endog[:,0]*self.endog[:,1]
        p10 = self.endog[:,0]* (1 - self.endog[:,1])
        p01 = (1 - self.endog[:,0]) * self.endog[:,1]

        ll = p00 * prob00 + p11 *prob11 + p01 * prob01 + p10 * prob10
        
       
        return -np.log(np.maximum(ll,1e-12)).sum()
        
        
    
    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        start_params = np.ones(self.exog.shape[1]+2)
        return super(NashLogit, self).fit(start_params=start_params,**kwds)

        
        
N = data.shape[0]
y = data[['y1','y2']]
x = data[['x1','x2']]

model = NashLogit(y,x)
model = model.fit()
print(model.summary(xname=["b11","d1","b12","d2"]))

KeyError: "None of [Index(['x1', 'x2'], dtype='object')] are in the [columns]"

In [9]:
# Modify the variables
entry_data = pd.read_csv("merged_entry.csv")
entry_data.loc[entry_data['HD'] > 0.0, 'HD_1'] = 1
entry_data.loc[entry_data['LO'] > 0.0, 'LO_1'] = 1
entry_data['income_2'] = np.log(1 + entry_data['income_per_capita'])
entry_data['population_2'] = np.log(1 + entry_data['population'])
entry_data['under44_1_2'] = np.log(1 + entry_data['under44_1'])
entry_data['under44_2_2'] = np.log(1 + entry_data['under44_2'])
entry_data['under44_3_2'] = np.log(1 + entry_data['under44_3'])
entry_data['older65_1_2'] = np.log(1 + entry_data['older65_1'])
entry_data['older65_2_2'] = np.log(1 + entry_data['older_65_2'])
entry_data = entry_data.fillna(0)

C:\Users\himan\anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [12]:
# Fit the model
N = entry_data.shape[0]
x = entry_data[['income_2','population_2','income_2','population_2']].copy()
y = entry_data[['HD_1','LO_1']]

model = NashLogit(y,x).fit()
model.summary()

C:\Users\himan\anaconda3\lib\site-packages\statsmodels\base\model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warn('Inverting hessian failed, no bse or cov_params '
C:\Users\himan\anaconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


<class 'statsmodels.iolib.summary.Summary'>
"""
                              NashLogit Results                               
==============================================================================
Dep. Variable:       ['HD_1', 'LO_1']   Log-Likelihood:                -4625.8
Model:                      NashLogit   AIC:                             9256.
Method:            Maximum Likelihood   BIC:                             9267.
Date:                Mon, 19 Apr 2021                                         
Time:                        10:23:05                                         
No. Observations:                2155                                         
Df Residuals:                    2153                                         
Df Model:                           1                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
income_2        -1.0573        nan        nan        nan         nan         nan
population_2     1.1026        nan        nan        nan         nan         nan
income_2         3.9061        nan        nan        nan         nan         nan
population_2     0.0097        nan        nan        nan         nan         nan
par0             0.0336        nan        nan        nan         nan         nan
par1             0.0274        nan        nan        nan         nan         nan
================================================================================
"""